In [46]:
# import library
import requests as rq
from bs4 import BeautifulSoup
import time
import pandas as pd
import logging
from pathlib import Path
import datetime

In [47]:
def sleeptime(hour, mins, secs):
    return hour*3600 + mins*60 + secs

In [48]:
logging.basicConfig(format='%(asctime)s: %(levelname)s: %(message)s')
logging.root.setLevel(level=logging.INFO)
logging.info("logging setted successfully")

2021-03-26 15:51:09,303: INFO: logging setted successfully


In [49]:
def extract_html_from_content(p_url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    p_response = rq.get(p_url, headers=headers)
    p_html_doc = p_response.text
    p_soup = BeautifulSoup(p_html_doc, "lxml")
    
    p_content = p_soup.find("div", "js-expand-project-content maxh7 mb3 overflow-hidden relative maxh-none-ns mv-child-0 overflow-visible-ns")
    p_content = p_content.prettify()
    
    return p_content

In [50]:
def insert_zec_project_into_mysql(sql_database, sql_table, sql_value):
    import mysql.connector
    
    # to connect the database
    sql_db = mysql.connector.connect(
        host = "127.0.0.1",
        user = "root",
        password = "",
        database = sql_database
    )
    # to check whether the TABLE exists or not
    cursor = sql_db.cursor()
    cursor.execute("SHOW TABLES LIKE '{}'".format(sql_table))
    temp_result = cursor.fetchone()
    if (temp_result):
        logging.info("TABLE {} exists in DATABASE {}".format(sql_table, sql_database))
    else:
        logging.info("NO TABLE {} in DATABASE {}".format(sql_table, sql_database))
        logging.info("ready to create TABLE {} in DATABASE {}".format(sql_table, sql_database))
        # to create a TABLE with PRIMARY KEY
        sql_create_table = "CREATE TABLE {} (id INT AUTO_INCREMENT PRIMARY KEY, inserted_date VARCHAR(255), project_name VARCHAR(255), project_title VARCHAR(255), project_type VARCHAR(255), category VARCHAR(255),  target_price INT, current_price INT, sponsor_amount INT, status VARCHAR(255), description_html MEDIUMTEXT, reward_item_amount INT)".format(sql_table)
        cursor.execute(sql_create_table)
        logging.info("created TABLE {} in DATABASE {}".format(sql_table, sql_database))
        
    # to insert a row of project data passed in
    sql_insert = "INSERT INTO {} (inserted_date, project_name, project_title, project_type, category, target_price, current_price, sponsor_amount, status, description_html, reward_item_amount) VALUES (NOW(), %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)".format(sql_table)
    cursor.execute(sql_insert, sql_value)
    sql_db.commit()
    logging.info("project {} INSERTED in TABLE {}".format(sql_value[1], sql_table)) # print out the project number that be inserted

# sql_database = "practice_db"
# sql_table = "v1_test_zeczec"

# sql_value = ("perzon-buds-anc", "【Perzon Buds | 真無線ANC主動降噪藍牙耳機】不鏽鋼工藝．聲入其境", "群眾集資", "科技", 100000, 363978, 130, "進行中", "<div class=js-expand-project-content maxh7 mb", 8)
# insert_zec_project_into_mysql(sql_database, sql_table, sql_value)

In [76]:
def get_zec_project_basic():
    t_start = time.time()
    
    all_p_df = pd.DataFrame()
    
    total_project_count = 0
    
    # explore url example: https://www.zeczec.com/categories?page=300&scope=new
    explore_domain = "https://www.zeczec.com/categories?page="
    page_count = 300
    link = explore_domain + str(page_count) + "&scope=new"
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = rq.get(link, headers=headers)
    html_doc = response.text
    soup = BeautifulSoup(html_doc, "lxml")
    # to get html of all the <div class="mb4 w-100 w-25-l w-50-m"> & store as a list
    projects = soup.find_all("div", "mb4 w-100 w-25-l w-50-m")
    # to check whether projectCards exist in the current explore page
    if (projects):
        logging.info("project cards exist on "+link)
    else:
        logging.info("project cards not exist on "+link)
    
    while (projects and page_count>0):
        
        # to count for current explore page
        count = len(projects)-1
        # to get the html of each project on the current explore page
        ###############################to control the number of project##############################
        while (count > -1):
        #while (count < 3):
            a_tag = projects[count].select_one("a.db")
            p_url = a_tag['href']
            p_domain = "https://www.zeczec.com"
            p_url = p_domain + p_url
            
            logging.info("now crawling project: "+p_url)
            
            # to get the status of the project
            check_ongoing = projects[count].select_one("i.material-icons.f6.v-mid.gray.mr1")
            if (check_ongoing):
                p_status = "進行中"
            else:
                check_end = projects[count].select_one("span.b")
                if (check_end):
                    check_end = check_end.text
                    if (check_end == "已發佈"):
                        p_status = "即將開始"
                    elif (check_end == "集資失敗"):
                        p_status = "失敗"
                    elif (check_end == "成功"):
                        p_status = "成功"
                    else:
                        p_status = "-"
                else:
                    p_status = "-"
            
            # to get the project's detail from the project html
            p_response = rq.get(p_url, headers=headers)
            p_html_doc = p_response.text
            p_soup = BeautifulSoup(p_html_doc, "lxml")
            p_detail = p_soup.find("div", "w-30-l w-100 ph3")
            
            # to check whether there is the elemnet on the current page
            # because if the product or the project is 18-forbidden
            # the element of p_detail would not exist on the checking page
            # the project can be junped over first
            if (p_detail):
                p_name = p_url.split('/')
                p_name = str(p_name[-1])
                p_title = p_detail.select_one("h2.f4.mt2.mb2").text.strip()
                p_type = p_detail.select_one("a.dark-gray.b.dib").text
                p_category = p_detail.select_one("a.gray.b.dib").text
                if (p_type == "訂閱式專案"): # there is no target money on subscribed project page
                    t_money = -1
                else: 
                    t_money = p_detail.select_one("div.f7.mt2").text.strip()
                    t_money = int(t_money[6:].replace(",", ""))
                now_money = p_detail.select_one("div.f3.b.js-sum-raised").text.strip()
                now_money = int(now_money[3:].replace(",", ""))
                #p_sponsor = int(p_detail.select_one("span.js-backers-count").text.strip())
                p_sponsor = p_detail.select_one("span.js-backers-count")
                if (p_sponsor):
                    p_sponsor = int(p_sponsor.text.strip())
                else:
                    p_sponsor = -1
                
                        
                p_content_html = extract_html_from_content(p_url)
                
                # to count the number of reward item & record
                content_html = p_soup.find("div", "container mv4")
                reward = content_html.find_all("div", "w-100-l w-50-ns ph3 ph0-l flex-none self-start")
                p_reward = len(reward)
                #logging.info(str(p_reward)+"= the number of reward item on the page: "+p_url)
                
                p_get_date = datetime.datetime.now()
                p_get_date = p_get_date.strftime('%Y-%m-%d %H:%M:%S')
                
                # to store the data into DataFrame
                temp_p_df = pd.DataFrame({
                    'inserted_date':p_get_date,
                    'project_name': p_name,
                    'project_title':p_title,
                    'project_type':p_type,
                    'project_category':p_category,
                    'target_price':t_money,
                    'current_price':now_money,
                    'sponsor_amount':p_sponsor,
                    'status': p_status,
                    'description_html':p_content_html,
                    'reward_item_amount':p_reward
                }, index=[total_project_count])
                
                all_p_df = pd.concat([all_p_df, temp_p_df])
                all_p_df = all_p_df.reset_index(drop=True)
                
                # to insert into TABLE & DATABASE
                sql_database = "crowdfunding_db"
                sql_table = "v3_latest_zeczec_basic_project_information"
                sql_value = (p_name, p_title, p_type, p_category, t_money, now_money, p_sponsor, p_status, p_content_html, p_reward)
                insert_zec_project_into_mysql(sql_database, sql_table, sql_value)
                
                total_project_count += 1
                
                logging.info("project "+p_name+" "+p_title+ p_url+" done")
                
                # change the sleeptime alternatively
                if (count%2 == 0):
                    break_secs = 3
                else:
                    break_secs = 10
                time.sleep(sleeptime(0, 0, break_secs))
                
            else: # there is no p_detail element on the current project page
                logging.info("there is no p_detail element on  the page: "+p_url)
            count -= 1 # whether there is p_detail element of not, just go to next project
        
        # to update the explore page
        logging.info("explore page: "+link+" done")
        # while page_count = 0, there is not "project card" on the explore_page
        # https://www.zeczec.com/categories?page=0&scope=new: 505
        page_count -= 1
        
        link = explore_domain + str(page_count) + "&scope=new"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = rq.get(link, headers=headers)
        html_doc = response.text
        soup = BeautifulSoup(html_doc, "lxml")
        # to get html of all the <div class="mb4 w-100 w-25-l w-50-m"> & store as a list
        projects = soup.find_all("div", "mb4 w-100 w-25-l w-50-m")
        # to check whether projectCards exist in the current explore page
        if (projects):
            logging.info("project cards exist on "+link)
        else:
            logging.info("project cards exist on "+link)
        
#         if (page_count == 50): # break for temp
#             break
    logging.info("Finished: {} projects are crawled".format(total_project_count))
    return all_p_df
    

            

In [77]:
# main block
def main():
    zec_p_df = get_zec_project_basic()
    print(zec_p_df)

if __name__=="__main__":
    main()

2021-03-29 13:20:22,658: INFO: project cards exist on https://www.zeczec.com/categories?page=300&scope=new
2021-03-29 13:20:22,660: INFO: now crawling project: https://www.zeczec.com/projects/no-more-future-no-more-past
2021-03-29 13:20:25,709: INFO: NO TABLE v3_latest_zeczec_basic_project_information in DATABASE crowdfunding_db
2021-03-29 13:20:25,711: INFO: ready to create TABLE v3_latest_zeczec_basic_project_information in DATABASE crowdfunding_db
2021-03-29 13:20:25,739: INFO: created TABLE v3_latest_zeczec_basic_project_information in DATABASE crowdfunding_db
2021-03-29 13:20:25,743: INFO: project 過去 X 未來 多提無用 手工絹印購物袋 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:20:25,744: INFO: project no-more-future-no-more-past 過去 X 未來 多提無用 手工絹印購物袋https://www.zeczec.com/projects/no-more-future-no-more-past done
2021-03-29 13:20:35,747: INFO: now crawling project: https://www.zeczec.com/projects/a-hundred-scenes-of-life
2021-03-29 13:20:38,153: INFO: TABLE v3_latest

2021-03-29 13:22:58,473: INFO: project 蕭芸安的城市聲音計畫2012 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:22:58,474: INFO: project the-friend-from-afar 蕭芸安的城市聲音計畫2012https://www.zeczec.com/projects/the-friend-from-afar done
2021-03-29 13:23:08,475: INFO: now crawling project: https://www.zeczec.com/projects/nomiirabbit-illustration-exhibition-at-tokyo
2021-03-29 13:23:11,113: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:23:11,117: INFO: project 糯米兔插畫展@東京 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:23:11,118: INFO: project nomiirabbit-illustration-exhibition-at-tokyo 糯米兔插畫展@東京https://www.zeczec.com/projects/nomiirabbit-illustration-exhibition-at-tokyo done
2021-03-29 13:23:14,119: INFO: now crawling project: https://www.zeczec.com/projects/techit
2021-03-29 13:23:20,018: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:23

2021-03-29 13:25:45,456: INFO: project millets-back-home 「好久不見德拉奇」紀錄片 - 回到初始地之部落首映會https://www.zeczec.com/projects/millets-back-home done
2021-03-29 13:25:48,458: INFO: now crawling project: https://www.zeczec.com/projects/bubbleblue
2021-03-29 13:25:51,412: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:25:51,418: INFO: project 【BUBBLE BLUE】秘密X夢 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:25:51,419: INFO: project bubbleblue 【BUBBLE BLUE】秘密X夢https://www.zeczec.com/projects/bubbleblue done
2021-03-29 13:26:01,422: INFO: now crawling project: https://www.zeczec.com/projects/daoyou
2021-03-29 13:26:04,025: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:26:04,025: INFO: project 來個島遊！-蒐集熱情計畫 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:26:04,025: INFO: project daoyou 來個島遊！-蒐集熱情計畫https://www.zeczec.com/projects/daoyou d

2021-03-29 13:28:41,877: INFO: project 《Holiday ya二手市集》再生x歡樂x社會企業 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:28:41,878: INFO: project holidayy2013 《Holiday ya二手市集》再生x歡樂x社會企業https://www.zeczec.com/projects/holidayy2013 done
2021-03-29 13:28:44,880: INFO: now crawling project: https://www.zeczec.com/projects/rogerspace
2021-03-29 13:28:47,359: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:28:47,362: INFO: project 【Roger Space】用籃球與插畫前進洛杉磯 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:28:47,363: INFO: project rogerspace 【Roger Space】用籃球與插畫前進洛杉磯https://www.zeczec.com/projects/rogerspace done
2021-03-29 13:28:57,364: INFO: now crawling project: https://www.zeczec.com/projects/violin
2021-03-29 13:28:59,987: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:28:59,990: INFO: project 打造電提琴的異想世界 INSERTED in TABLE v3_latest_ze

2021-03-29 13:31:19,371: INFO: project 2014《我的未來就是夢？》音樂節 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:31:19,371: INFO: project 2014-2015 2014《我的未來就是夢？》音樂節https://www.zeczec.com/projects/2014-2015 done
2021-03-29 13:31:29,372: INFO: now crawling project: https://www.zeczec.com/projects/findyourdeer
2021-03-29 13:31:31,917: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:31:31,917: INFO: project 國際論壇「遺失ㄌㄨˋ中心」 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 13:31:31,917: INFO: project findyourdeer 國際論壇「遺失ㄌㄨˋ中心」https://www.zeczec.com/projects/findyourdeer done
2021-03-29 13:31:34,927: INFO: now crawling project: https://www.zeczec.com/projects/sunflowerphoto
2021-03-29 13:31:37,570: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 13:31:37,570: INFO: project 太陽花學運記錄攝影集 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2

...

2021-03-29 22:59:20,315: INFO: now crawling project: https://www.zeczec.com/projects/lucky-cat-dice-game
2021-03-29 22:59:23,219: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 22:59:23,219: INFO: project 萌萌貓咪大作戰 ！幸運貓桌遊 Lucky Cat Dice Game INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 22:59:23,219: INFO: project lucky-cat-dice-game 萌萌貓咪大作戰 ！幸運貓桌遊 Lucky Cat Dice Gamehttps://www.zeczec.com/projects/lucky-cat-dice-game done
2021-03-29 22:59:33,227: INFO: now crawling project: https://www.zeczec.com/projects/the-a-bc-card-games
2021-03-29 22:59:35,986: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 22:59:36,002: INFO: project 三八弟桌遊｜The A+BC Card Games【學英文的新方法】邊玩邊背單字　多種遊戲讓小朋友開心學習 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 22:59:36,002: INFO: project the-a-bc-card-games 三八弟桌遊｜The A+BC Card Games【學英文的新方法】邊玩邊背單字　多種遊戲讓小朋友開心學習https://w

2021-03-29 23:02:03,208: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 23:02:03,210: INFO: project 第六屆政大音樂節 ⟪換日修復計畫⟫｜全台唯一結合新媒體藝術的學生音樂節 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 23:02:03,210: INFO: project NCCU_Music_Festival_2021 第六屆政大音樂節 ⟪換日修復計畫⟫｜全台唯一結合新媒體藝術的學生音樂節https://www.zeczec.com/projects/NCCU_Music_Festival_2021 done
2021-03-29 23:02:13,212: INFO: now crawling project: https://www.zeczec.com/projects/renyou
2021-03-29 23:02:17,622: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 23:02:17,625: INFO: project 幫憨兒找個家｜仁友愛心家園搬遷計畫：重建家園 讓愛蔓延！ INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 23:02:17,625: INFO: project renyou 幫憨兒找個家｜仁友愛心家園搬遷計畫：重建家園 讓愛蔓延！https://www.zeczec.com/projects/renyou done
2021-03-29 23:02:20,641: INFO: now crawling project: https://www.zeczec.com/projects/level99
2021-03-29 23:02:23,926: INFO: TABLE v3_l

2021-03-29 23:05:01,711: INFO: project Bubble 磁吸式寵物碗，瞬間吸合！毛小孩的用餐高度由你來決定 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 23:05:01,711: INFO: project bubblepetbowl Bubble 磁吸式寵物碗，瞬間吸合！毛小孩的用餐高度由你來決定https://www.zeczec.com/projects/bubblepetbowl done
2021-03-29 23:05:04,722: INFO: now crawling project: https://www.zeczec.com/projects/ngiyas-tw-glass
2021-03-29 23:05:08,842: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 23:05:08,842: INFO: project NGIYAS 玻璃杯 ☆ 越夜越美的植物杯 ☆ INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 23:05:08,842: INFO: project ngiyas-tw-glass NGIYAS 玻璃杯 ☆ 越夜越美的植物杯 ☆https://www.zeczec.com/projects/ngiyas-tw-glass done
2021-03-29 23:05:18,855: INFO: now crawling project: https://www.zeczec.com/projects/chu-lu-guanmiao
2021-03-29 23:05:21,694: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 23:05:21,694: INFO: project 《竹鹿關廟

2021-03-29 23:07:50,691: INFO: now crawling project: https://www.zeczec.com/projects/USii_lovekids_rainjacket
2021-03-29 23:07:54,557: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 23:07:54,557: INFO: project 【用最好呵護最愛  皮膚科醫師推薦】 極輕量  超快乾  透氣兒童防水外套 INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 23:07:54,572: INFO: project USii_lovekids_rainjacket 【用最好呵護最愛  皮膚科醫師推薦】 極輕量  超快乾  透氣兒童防水外套https://www.zeczec.com/projects/USii_lovekids_rainjacket done
2021-03-29 23:08:04,574: INFO: now crawling project: https://www.zeczec.com/projects/drinkhaha
2021-03-29 23:08:07,504: INFO: TABLE v3_latest_zeczec_basic_project_information exists in DATABASE crowdfunding_db
2021-03-29 23:08:07,504: INFO: project 『不想上班，只想笑喝喝！』新型態酒類桌遊，成為你喝酒時的最佳配備！ INSERTED in TABLE v3_latest_zeczec_basic_project_information
2021-03-29 23:08:07,504: INFO: project drinkhaha 『不想上班，只想笑喝喝！』新型態酒類桌遊，成為你喝酒時的最佳配備！https://www.zeczec.com/projects/drinkhaha done
2

            inserted_date                 project_name  \
0     2021-03-29 13:20:25  no-more-future-no-more-past   
1     2021-03-29 13:20:38     a-hundred-scenes-of-life   
2     2021-03-29 13:20:44                      haniboi   
3     2021-03-29 13:20:57        victim-fashion-street   
4     2021-03-29 13:21:02           the-bravery-course   
...                   ...                          ...   
3572  2021-03-29 23:07:28                     LTB_Halo   
3573  2021-03-29 23:07:34                  pSc-oriolis   
3574  2021-03-29 23:07:47                 smile-makers   
3575  2021-03-29 23:07:54     USii_lovekids_rainjacket   
3576  2021-03-29 23:08:07                    drinkhaha   

                                     project_title project_type  \
0                             過去 X 未來 多提無用 手工絹印購物袋         群眾集資   
1                         掙扎人生，無謂人生，生活中間的 < 人森百回 >         群眾集資   
2                                          李翰的內個計畫         群眾集資   
3     Victim Fashion Street: Interl